# 泛型

## 1. 概述

Java泛型是J2SE1.5中引入的一个新特性，其本质是参数化类型，也就是说所操作的数据类型被指定为一个参数（type parameter）这种参数类型可以用在类、接口和方法的创建中，分别称为泛型类、泛型接口、泛型方法。

## 2. 没有泛型会怎样


In [3]:
List arrayList = new ArrayList();
// 可以向arrayList中加入任意类型的对象
arrayList.add("string");
arrayList.add(3.14);

for(int i = 0; i< arrayList.size();i++){
    // 在取值时做强制转换
    String item = (String)arrayList.get(i);
    System.err.println("泛型测试, item = " + item);
}

泛型测试, item = string


EvalException: class java.lang.Double cannot be cast to class java.lang.String (java.lang.Double and java.lang.String are in module java.base of loader 'bootstrap')

毫无疑问，程序的运行结果会以崩溃结束

`ArrayList`可以存放任意类型，例子中添加了一个`String`类型，添加了一个`Double`类型，再使用时都以`String`的方式使用，因此程序崩溃了。为了解决类似这样的问题（在编译阶段就可以解决），泛型应运而生。

我们将第一行声明初始化list的代码更改一下，编译器会在编译阶段就能够帮我们发现类似这样的问题。

In [4]:
List<String> arrayList = new ArrayList<>();
arrayList.add("string");
arrayList.add(3.14); // 编译器报错，你不能加入 double 类型的数据

CompilationException: 

## 3. 泛型只在编译阶段有效

泛型只在编译阶段有效，即编译后还是没有泛型，泛型只是语法糖。看下面的代码：

In [6]:
List<String> stringArrayList = new ArrayList<String>();
List<Integer> integerArrayList = new ArrayList<Integer>();
System.out.println("stringArrayList.getClass() == integerArrayList.getClass() ? " 
    + (stringArrayList.getClass() == integerArrayList.getClass()));

stringArrayList.getClass() == integerArrayList.getClass() ? true


执行，你会发现比较相等的结果是 **true**

## 4. 泛型的使用

### 4.1 泛型类

一个泛型类，就是具有一个或多个类型变量的类。我们使用一个简单的Pair类作为例子。

In [8]:
public class Pair<T> {
    private T first;
    private T second;

    public Pair() {
    }

    public Pair(T first, T second) {
        this.first = first;
        this.second = second;
    }

    public T getFirst() {
        return first;
    }

    public void setFirst(T first) {
        this.first = first;
    }

    public T getSecond() {
        return second;
    }

    public void setSecond(T second) {
        this.second = second;
    }
}

Pair类引入了一个类型变量T，使用尖括号（<>）括起来，放在类名的后面。  
泛型类可以引入多个类型变量，例如，Pair的第一个实例变量和第二个实例变量的类型可以不同:
```Java
public class Pair<T, U> { }
```
类定义中的**类型变量**可以用来定义**实例变量, 方法的返回值类型，方法参数，局部变量**
```Java
private T first;
```

**注意**:
> 类型变量使用大写字母，并且比较短。一般的，在Java库中:
> - E表示集合的元素类型，
> - K和V分别表示哈希表的关键字和值的类型
> - T(或U, S)表示任意类型

使用Pair<T>时，用具体的类型替换类型变量实例化泛型类，例如：
```Java
Pair<String>
```
可以将结果想象成如下的普通类
```Java
Pair<String>()
String getFirst()
void setFirst(String)
```

下面的代码使用`Pair`类，静态方法`minmax`找出数组中的最大、最小值，用`Pair`对象返回两个结果。

In [9]:
public class ArrayAlg {
    // 得到数组a中的最大值和最小值
    public static Pair<String> minmax(String[] a) {
        if (a==null || a.length==0) {
            return null;
        }
        String min = a[0];
        String max = a[0];
        for (String s : a) {
            if (min.compareTo(s) > 0) {
                min = s;
            }
            if (max.compareTo(s) < 0) {
                max = s;
            }
        }
        return new Pair<>(min, max);
    }
}

In [10]:
public class PairTest1 {
    public static void main(String[] args) {
        String[] words = {"Mary", "had", "a", "little", "lamb"};
        Pair<String> mm = ArrayAlg.minmax(words);
        System.out.println("min = " + mm.getFirst());
        System.out.println("max = " + mm.getSecond());
    }
}

In [12]:
// 执行 PairTest1
PairTest1.main(null);

min = Mary
max = little


### 4.2 泛型方法

泛型方法可以在泛型类中定义，也可以在普通类中。   
如下，定义了一个泛型方法`getMiddle`

In [13]:
public class ArrayAlg {
    // 得到数组中的中间值
    public static <T> T getMiddle(T... a) {
        return a[a.length / 2];
    }
}

In [14]:
// 当调用泛型方法时，在方法前的<>中放入具体类型
String m1 = ArrayAlg.<String>getMiddle("John", "Q", "Public");
System.out.println("m1 = " + m1);
// 也可以省略<String>类型参数，编译器能推断出所调用的方法
String m2 = ArrayAlg.getMiddle("John", "Q", "Public");
System.out.println("m2 = " + m2);

m1 = Q
m2 = Q


但有时候编译器也会提示推断也会出错，如下的代码编译器就不能正确推断
```Java
double m3 = ArrayAlg.getMiddle(3.14, 100, 0);
```

### 4.3 变量类型的限定

有时，类或方法需要对类型的变量加以约束。例如，我们需要得到数组中的最小元素
```Java
public class ArrayAlg {
    public static <T> T min(T[] a) {
        if (a == null || a.length == 0) {
            return null;
        }
        T min = a[0];
        for (T v : a) {
            if (min.compareTo(v) > 0) {
                min = v;
            }
        }
        return min;
    }
}
```
代码有一个问题，变量`min`的类型为T，意味着它可以是任意一种类型，怎么才能确保T所属的类有compareTo方法呢？   
解决的办法就是将T限制为实现了`Comparable`接口的类，如下所示。

In [15]:
public class ArrayAlg {
    public static <T extends Comparable> T min(T[] a) {
        if (a == null || a.length == 0) {
            return null;
        }
        T min = a[0];
        for (T v : a) {
            if (min.compareTo(v) > 0) {
                min = v;
            }
        }
        return min;
    }
}

In [19]:
import java.time.LocalDate;
public class PairTest3 {
    public static void main(String[] args) {
        LocalDate[] birthdays = {
                LocalDate.of(1919, 5, 4),
                LocalDate.of(1927, 8, 1),
                LocalDate.of(1949, 10, 1),
        };
        LocalDate min = ArrayAlg.min(birthdays);
        System.out.println("min = " + min);
    }
}

In [20]:
// 执行
PairTest3.main(null);

min = 1919-05-04


类型变量限制语法：
```Java
<T extends BoundingType>
```
表示`T`应该是`BoundingType`的子类型，`T`和`BoundingType`可以是类，也可以是接口，选择关键字`extends`的原因是更接近子类的概念。  
一个类型变量可以有多个限定，例如：
```Java
T extends Comarable & Serializable
```
限定类型用&分开，而逗号用来分隔类型变量。

### 4.4 泛型使用约束和局限性

#### 4.4.1 不能使用primitive主数据类型
    不能使用primitive主数据类型。因此，没有`Pair<double>`, 只有`Pair<Double>`。

#### 4.4.2 不能创建初始化类型的数组
```Java
Pair<String>[] aa = new Pair<String>[10]; // error
```
但可以使用ArrayList
```Java
ArrayList<Pair<String>> list = new ArrayList<>();
```

#### 4.4.3 不能实例化类型变量
    不能使用`new T(), new T[]`这样的代码
```Java
public Pair() { first = new T(); second = new T(); } // error
```
要这样:
```Java
public static <T> Pair<T> makePair(Class<T> clazz) {
    try {
        return new Pair<>(clazz.newInstance(),clazz.newInstance());
    } catch (Exception e) {
        e.printStackTrace();
        return null;
    }
}
```

### 4.5 泛型类型的继承规则

回忆之前学过的动物继承树

In [21]:
public abstract class Animal {
    public abstract void sleep();
    public abstract void roam();
    public abstract void eat();
    public abstract void makeNoise();
}

In [22]:
public class Dog extends Animal {
    @Override
    public void sleep() {
        System.out.println("Dog sleep");
    }

    @Override
    public void roam() {
        System.out.println("Dog roam");
    }

    @Override
    public void eat() {
        System.out.println("Dog eat");
    }

    @Override
    public void makeNoise() {
        System.out.println("Dog makeNoise");
    }
}

思考问题，`Pair<Animal>`和`Pair<Dog>`是什么关系，你肯定以为`Pair<Dog>`是`Pair<Animal>`的子类，但并不是，`Pair<Animal>`和`Pair<Dog>`**没有继承关系**。
下面的代码是错误的：

In [23]:
Pair<Dog> ds = new Pair();
Pair<Animal> as = ds;

CompilationException: 

 ## 4.6 泛型通配符
 
为了解决上面的问题，Java设计者发明了一种巧妙的“解决方案”：通配符类型。

### 4.6.1 通配符子类限定

通配符类型中，允许参数变化，例如：
```Java
Pair<? extends Animal>
```
表示任何泛型Pair类型，它的类型参数是`Animal`的子类，如`Pair<Dog>`，但不是`Pair<String>`。

例题：假设有一个动物园类`Zoo`，可以成批量添加动物。

In [25]:
public class Zoo {
    ArrayList<Animal> list = new ArrayList<>();

    // 向动物园中添加动物
    public void addAnimals(List<? extends Animal> as) {
        for (Animal a : as) {
            list.add(a);
        }
    }

    // 动物数量
    public int animalCount() {
        return list.size();
    }
}

In [26]:
public class ZooTest {
    public static void main(String[] args) {
        Zoo zoo = new Zoo();
        System.out.println("before add animal, count = " + zoo.animalCount());
        // 创建dog list 放入两只狗
        List<Dog> dogs = Arrays.asList(new Dog(), new Dog());
        // 加到动物园
        zoo.addAnimals(dogs);
        System.out.println("after add animal, count = " + zoo.animalCount());
    }
}

In [27]:
// 执行
ZooTest.main(null);

before add animal, count = 0
after add animal, count = 2


注意：使用`List<? extends Animal>`定义的变量，不能再加入元素，考虑如下的代码
```Java
List<? extends Animal> as = new ArrayList<Dog>();
as.add(new Wolf()); // 向ArrayList<Dog>中加入Wolf对象是错误的
```

### 4.6.1 通配符超类限定


超类限定和子类限定类似，如下：
```Java
? super Dog
```
这个通配符限制为`Dog`的所有超类，它的作用是可以写入值，但不能读取值。如下的代码
```Java
List<? super Animal> as = new ArrayList<Animal>();
as.add(new Dog()); // 正确
as.add(new Wolf()); // 正确
Animal a = as.get(0); // 错误
```

回顾前面的求最小值的方法头部
```Java
public static <T extends Comparable> T min(T[] a);
```
这里`Comparable`本身就是一个泛型接口，接口定义如下:
```Java
public interface Comaprable<T> {
    public int compareTo(T other);
}
```
也许，我们可以把`min`方法的头部修改为:
```Java
public static <T extends Comparable<T>> T min(T[] a);
```
但修改后发现，原有的代码有语法错误
```Java
LocalDate min = ArrayAlg.min(birthdays);
```
这是因为，`LocalDate`实现了`ChronoLocalDate`, 而`ChronoLocalDate`扩展了`Comparable<ChronoLocalDate>`，因此`LocalDate`实现的是`Comparable<ChronoLocalDate>`而不是`Comparable<LocalDate>`   

为解决这个问题，可以使用超类限定   
```Java
public static <T extends Comparable<? super T>> T min(T[] a)
```

### 通配符限定小节

- 带有超类限定的通配符可以向泛型对象写入
- 带有子类限定的通配符可以从泛型对象读取


### 作业

实现下面的方法
```Java
public class ArrayAlg {
    // 作业，实现此泛型方法
    public static <T extends Comparable<? super T>> Pair<T> minmax(T[] a) {
        return null;
    }
}
```
要求：运行通过单元测试`UTminmax`